In [28]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.optim as optim
import pickle

In [64]:
def resample_and_interpolate(group):
    return group.reset_index(level=[1, 2]).resample('30T').asfreq().interpolate()

In [65]:
weather_df = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/weather_data/DWD_ICON-EU.csv')
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
weather_df.sort_values(by='ref_datetime', inplace=True)
weather_df = weather_df.groupby(["valid_datetime","latitude","longitude"]).last().reset_index()
weather_df.reset_index(inplace=True)

In [ ]:
weather_df.valid_datetime = pd.to_datetime(weather_df.valid_datetime)
weather_df = weather_df.set_index(["valid_datetime","latitude","longitude"])
df_resampled = weather_df.groupby(['latitude', 'longitude'], group_keys=False).apply(resample_and_interpolate)
df_resampled = df_resampled.reset_index()

In [67]:
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)
df_resampled.drop(columns=['index','ref_datetime'], inplace=True)
df_resampled_merged = pd.merge(df_resampled, solar_total, how='left', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_solar = df_resampled_merged.loc[~(df_resampled_merged.latitude == 53.935) & ~(df_resampled_merged.longitude == 1.8645)]
df_resampled_merged_solar1 = df_resampled_merged_solar.groupby("valid_datetime").mean().reset_index()
distinct_lat_lon_pairs = df_resampled_merged_solar[['latitude', 'longitude']].drop_duplicates()

In [68]:
df_resampled_merged_wind = pd.merge(df_resampled, wind_total, how='inner', left_on='valid_datetime', right_on='timestamp_utc')
df_resampled_merged_wind = df_resampled_merged_wind.loc[(df_resampled_merged_wind.latitude == 53.935) & (df_resampled_merged_wind.longitude == 1.8645)]
df_resampled_merged_wind.drop_duplicates(inplace=True)
df_resampled_merged_wind


,valid_datetime,latitude,longitude,Temperature,WindSpeed,WindSpeed:100,WindDirection:100,CloudCover,RelativeHumidity,PressureReducedMSL,SolarDownwardRadiation,TotalPrecipitation,timestamp_utc,settlement_date,settlement_period,boa,generation_mw
40,2024-10-07 00:00:00+00:00,53.935,1.8645,14.930556,8.763889,11.313333,159.952778,1.000000,93.232778,99385.651667,0.0,NaN,2024-10-07 00:00:00+00:00,2024-10-07,3,0.0,482.321
62,2024-10-07 00:30:00+00:00,53.935,1.8645,14.891944,9.500139,12.232361,162.957083,1.000000,94.189028,99358.019583,0.0,NaN,2024-10-07 00:30:00+00:00,2024-10-07,4,0.0,541.471
83,2024-10-07 01:00:00+00:00,53.935,1.8645,14.853333,10.236389,13.151389,165.961389,1.000000,95.145278,99330.387500,0.0,1.614722,2024-10-07 01:00:00+00:00,2024-10-07,5,0.0,569.911
124,2024-10-07 01:30:00+00:00,53.935,1.8645,14.865694,9.685278,12.327083,184.986806,1.000000,94.389861,99286.175278,0.0,0.876389,2024-10-07 01:30:00+00:00,2024-10-07,6,0.0,568.161
146,2024-10-07 02:00:00+00:00,53.935,1.8645,14.878056,9.134167,11.502778,204.012222,1.000000,93.634444,99241.963056,0.0,0.138056,2024-10-07 02:00:00+00:00,2024-10-07,7,0.0,568.211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,2024-10-08 19:30:00+00:00,53.935,1.8645,15.107778,4.958472,7.572917,178.350000,0.371111,88.645139,99239.545000,0.0,0.000417,2024-10-08 19:30:00+00:00,2024-10-08,42,0.0,395.712
2015,2024-10-08 20:00:00+00:00,53.935,1.8645,15.105000,5.028611,7.633056,178.156667,0.537778,88.688611,99231.603333,0.0,0.000000,2024-10-08 20:00:00+00:00,2024-10-08,43,0.0,291.852
2036,2024-10-08 20:30:00+00:00,53.935,1.8645,15.093194,5.382917,8.030694,174.153750,0.768889,88.485833,99228.149583,0.0,0.013333,2024-10-08 20:30:00+00:00,2024-10-08,44,0.0,282.192
2057,2024-10-08 21:00:00+00:00,53.935,1.8645,15.081389,5.737222,8.428333,170.150833,1.000000,88.283056,99224.695833,0.0,0.026667,2024-10-08 21:00:00+00:00,2024-10-08,45,0.0,255.792


In [70]:
def set_up_wind_features(df):
    R_d = 287.05  # Specific gas constant for dry air (J/(kg·K))
    R_v = 461.5   # Specific gas constant for water vapor (J/(kg·K))
    p = 101325    # Standard atmospheric pressure in Pa
    # Calculate saturation vapor pressure (using temperature in Celsius), Tetens formula
    df['Temperature_K'] = df['Temperature'] + 273.15
    e_s = 0.61078 * np.exp((17.27 * (df['Temperature'])) / (df['Temperature'] +237.3))
    # in pa
    e_s = 1000 * e_s
    # Calculate actual vapor pressure
    e = df['RelativeHumidity'] / 100 * e_s
    df['AirDensity'] = (p - e) / (R_d * df['Temperature_K']) + (e / (R_v * df['Temperature_K']))
    # Turbine stats
    rotor_diameter = 154  # in meters
    approximated_total_efficiency = 0.348
    limiter = 0.94
    minimum_wind_speed = 3  # in m/s
    maximum_wind_speed_for_power_curve = 12.5  # in m/s
    maximum_wind_speed_for_operation = 25  # in m/s
    rotor_area = np.pi * (rotor_diameter / 2) ** 2  # in m²
    # turbine requires 3m/s to start rotating
    const_internal_friction_coefficient = 0.5 * 1.240 * np.pi * 77**2 * 3**3 * approximated_total_efficiency * 174 / 1000000
    maximum_power_per_turbine = 7 # in MW
    # Same for full
    df['WindSpeed_full_avg'] = (df['WindSpeed'] + df['WindSpeed:100']) / 2
    df['WindPower_full'] = 0.5 * df['AirDensity'] * rotor_area * df['WindSpeed:100'] ** 3 * 174 / 1000000
    df['UsableWindPower_full'] = np.minimum(df['WindPower_full'], maximum_power_per_turbine * 174 * limiter / approximated_total_efficiency)
    df['PowerOutput_full'] = np.where((df['WindSpeed:100'] >= minimum_wind_speed) & (df['WindSpeed:100'] <= maximum_wind_speed_for_operation), df['UsableWindPower_full'] * approximated_total_efficiency - const_internal_friction_coefficient, 0)

    # wind_df["Temperature_avg"] = (wind_df["Temperature"] + wind_df["Temperature:100"]) / 2
    # wind_df["RelativeHumidity_avg"] = (wind_df["RelativeHumidity"] + wind_df["RelativeHumidity:100"]) / 2
    df["Temperature_avg"] = df["Temperature"]
    df["RelativeHumidity_avg"] = df["RelativeHumidity"]   
    df["WindSpeed:100_dwd_lag1"] = df["WindSpeed:100"].shift(1)
    df["WindSpeed:100_dwd_lag2"] = df["WindSpeed:100"].shift(2)
    df["WindSpeed:100_dwd_lag3"] = df["WindSpeed:100"].shift(3)
    df["UsableWindPower_opt"] = df.UsableWindPower_full
    df["WindSpeed:100_dwd"] = df["WindSpeed:100"].shift(1)
    return df

In [74]:
df_resampled_merged_wind_2 = set_up_wind_features(df_resampled_merged_wind)
df_resampled_merged_wind_2.dropna(inplace=True)
X_wind = df_resampled_merged_wind_2[['WindSpeed:100_dwd', 'Temperature_avg', 'RelativeHumidity_avg', 'AirDensity',"UsableWindPower_opt", 'WindSpeed:100_dwd_lag1', 'WindSpeed:100_dwd_lag2', 'WindSpeed:100_dwd_lag3']]
y_wind = df_resampled_merged_wind_2['generation_mw']

In [33]:
def set_up_solar_features(df):
    df["hour"] = df.valid_datetime.dt.hour
    df["day_of_year"] = df.valid_datetime.dt.dayofyear
    df["cos_day_of_year"] = np.cos(2 * np.pi * df.day_of_year / 365)
    df["cos_hour"] = np.cos(2 * np.pi * df.hour / 24)
    df["Mean_SolarDownwardRadiation"] = df.SolarDownwardRadiation
    df["Mean_Temperature"] = df.Temperature
    df["Std_Temperature"] = df_resampled_merged_solar.groupby("valid_datetime").std().reset_index().Temperature
    df["SolarDownwardRadiation_RW_Mean_30min"] = df.Mean_SolarDownwardRadiation.rolling(window=1, min_periods=1).mean()
    df["SolarDownwardRadiation_RW_Mean_1hour"] = df.Mean_SolarDownwardRadiation.rolling(window=2, min_periods=1).mean()
    df["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df.Mean_SolarDownwardRadiation.shift(1)
    df["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df.Mean_SolarDownwardRadiation.shift(2)
    df["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df.Mean_SolarDownwardRadiation.shift(48)
    for i in range(len(distinct_lat_lon_pairs)):
        lat = distinct_lat_lon_pairs.latitude.iloc[i]
        lon = distinct_lat_lon_pairs.longitude.iloc[i]
        mask = (df_resampled_merged_solar.latitude == lat) & (df_resampled_merged_solar.longitude == lon)
        df[f"Temperature_{i}"] = pd.Series(df_resampled_merged_solar.Temperature[mask].values)[:len(df)]  # Fill gaps with NaN
        df[f"SolarDownwardRadiation_{i}"] = pd.Series(df_resampled_merged_solar.SolarDownwardRadiation[mask].values)[:len(df)]  # Fill gaps with NaN
    return df
df_resampled_merged_solar2 = set_up_solar_features(df_resampled_merged_solar1)

In [34]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [35]:
for i in range(20):
    temp_col = f'Temperature_{i}'
    irradiance_col = f'SolarDownwardRadiation_{i}'
    panel_temp_col = f'Panel_Temperature_Point{i}'
    panel_eff_col = f'Panel_Efficiency_Point{i}'
    df_resampled_merged_solar2[panel_temp_col], df_resampled_merged_solar2[panel_eff_col] = pv_temperature_efficiency(df_resampled_merged_solar2[irradiance_col], df_resampled_merged_solar2[temp_col])
df_resampled_merged_solar2["Panel_Temperature_dwd_mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_dwd_mean"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").mean(axis= 1)
df_resampled_merged_solar2["Panel_Temperature_dwd_std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Temperature.*").std(axis= 1)
df_resampled_merged_solar2["Panel_Efficiency_dwd_std"] = df_resampled_merged_solar2.filter(regex= r"Panel_Efficiency.*").std(axis= 1)
df_resampled_merged_solar2["solar_mw_lag_48h"] = df_resampled_merged_solar2.generation_mw.shift(periods= 96)
df_resampled_merged_solar2["capacity_mwp_lag_48h"] = df_resampled_merged_solar2.capacity_mwp.shift(periods= 96)
df_resampled_merged_solar2["Target_Capacity_MWP%"] = df_resampled_merged_solar2.generation_mw / df_resampled_merged_solar2.capacity_mwp
df_resampled_merged_solar2["Target_Capacity_MWP%_lag_48h"] = df_resampled_merged_solar2["Target_Capacity_MWP%"].shift(periods= 96)


C:\Users\paulh\AppData\Local\Temp\ipykernel_5056\3533751230.py:14: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [36]:
df_resampled_merged_solar3 = df_resampled_merged_solar2[[ 
    "Mean_SolarDownwardRadiation",
    "SolarDownwardRadiation_RW_Mean_1hour",
    "SolarDownwardRadiation_RW_Mean_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature",
    "Mean_Temperature",
    "cos_hour",
    "cos_day_of_year",
    "solar_mw_lag_48h",
    "capacity_mwp_lag_48h",
    "Target_Capacity_MWP%_lag_48h",
    "Target_Capacity_MWP%"
    ]]
df_resampled_merged_solar3.dropna(inplace=True)
Y_solar = df_resampled_merged_solar3["Target_Capacity_MWP%"]
X_solar = df_resampled_merged_solar3.drop(columns=["Target_Capacity_MWP%"])
X_solar.capacity_mwp_lag_48h.mean()

C:\Users\paulh\AppData\Local\Temp\ipykernel_5056\3836400376.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2779.3337282577586

In [37]:
def modified_pinball_loss(y_true, y_pred, quantile):
    delta = y_true - y_pred
    return np.mean(np.maximum(quantile * delta, (quantile - 1) * delta))

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor  # Dies ist nur für die Typisierung notwendig

In [81]:
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.exceptions import NotFittedError

In [83]:
import joblib
def load_pickle1(path):
    return joblib.load(path)

In [90]:
y_wind

272     511.971
293     557.040
314     556.171
335     550.081
356     545.471
         ...   
1994    395.712
2015    291.852
2036    282.192
2057    255.792
2078    240.632
Name: generation_mw, Length: 83, dtype: float64

In [92]:
predictions_wind.shape

(83,)

In [98]:
df_resampled_merged_wind_2[['WindPower_full','UsableWindPower_full','PowerOutput_full']]

,WindPower_full,UsableWindPower_full,PowerOutput_full
272,4310.594623,3290.000000,1126.039423
293,4749.057936,3290.000000,1126.039423
314,4438.878700,3290.000000,1126.039423
335,4142.463878,3290.000000,1126.039423
356,4030.903134,3290.000000,1126.039423
...,...,...,...
1994,856.929085,856.929085,279.330744
2015,877.513956,877.513956,286.494279
2036,1021.982377,1021.982377,336.769290
2057,1181.502216,1181.502216,392.282194


In [107]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.exceptions import InconsistentVersionWarning
import warnings

# Suppress the version mismatch warning
warnings.filterwarnings("ignore", category=InconsistentVersionWarning)
def modified_pinball_loss(y_true, y_pred, quantile):
    error = y_true - y_pred
    return np.mean(np.maximum(quantile * error, (quantile - 1) * error))

path = 'D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Wind_forecast/models/gbr_quantile_0.'
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
losses = []

for quantile in quantiles:
    with open(f"{path}{quantile}_res-True_calc-False.pkl", "rb") as f:
        model_wind = load_pickle1(f)
    
    if not hasattr(model_wind, '_preprocessor'):
        model_wind._preprocessor = None

    predictions_wind = model_wind.predict(X_wind)
    df_resampled_merged_wind_2[f"generation_mw_quantile_{quantile}"] = predictions_wind + df_resampled_merged_wind_2.PowerOutput_full /2
    loss = modified_pinball_loss(y_wind, df_resampled_merged_wind_2[f"generation_mw_quantile_{quantile}"], quantile/10)
    losses.append(loss)
    print(f"Quantile {quantile} loss: {loss}")


Quantile 1 loss: 20.195999526547254
Quantile 2 loss: 33.4644348745386
Quantile 3 loss: 41.2747916668606
Quantile 4 loss: 48.93325275958429
Quantile 5 loss: 55.02364323642375
Quantile 6 loss: 55.83523508717094
Quantile 7 loss: 47.65765468159276
Quantile 8 loss: 36.374967503132005
Quantile 9 loss: 20.2480305906291


In [106]:
import plotly.graph_objects as go

# Create an empty figure
fig = go.Figure()

# Plot actual values
fig.add_trace(go.Scatter(
    x=df_resampled_merged_wind_2.index,  # Assuming the index is time or some sequence
    y=y_wind,  # Actual values
    mode='lines',
    name='Actual Generation',
    line=dict(color='black', width=2)  # Style for actual values
))

# Plot predicted values for each quantile
for quantile in quantiles:
    fig.add_trace(go.Scatter(
        x=df_resampled_merged_wind_2.index,  # Assuming same x-axis for predictions
        y=df_resampled_merged_wind_2[f'generation_mw_quantile_{quantile}'],  # Predictions for each quantile
        mode='lines',
        name=f'Quantile {quantile} Prediction',
        line=dict(width=1.5)  # Style for predicted values
    ))

# Update layout for better visualization
fig.update_layout(
    title='Actual vs Predicted Wind Power Generation',
    xaxis_title='Time',
    yaxis_title='Power Generation (MW)',
    legend_title='Legend',
    hovermode='x',
    template='plotly_white'
)

# Show the plot
fig.show()


In [41]:
path = "D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/models/lgbr_model/models/i5_models/lgbr_q"
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
losses = []

# Berechnung der Verluste für jedes Quantil
for i in quantiles:
    model_light = pickle.load(open(path + str(i) + ".pkl", 'rb'))
    predictions = model_light.predict(X_solar)
    loss = modified_pinball_loss(Y_solar.values, predictions, i/10)
    losses.append(loss)
    print(f"Quantile {i/10}: {loss*2779.3337282577586}")

Quantile 0.1: 4.886879487391104
Quantile 0.2: 7.09266779824093
Quantile 0.3: 9.157585428209972
Quantile 0.4: 9.519838945762436
Quantile 0.5: 9.327083079658887
Quantile 0.6: 9.607099934046316
Quantile 0.7: 9.138098901895782
Quantile 0.8: 8.782410076969102
Quantile 0.9: 4.911396093562053


In [39]:
import plotly.express as px
# Erstellen eines DataFrames für Plotly
df = pd.DataFrame({
    'Quantile': [i/10 for i in quantiles],
    'Loss': losses
})

# Plotten der Verluste mit Plotly
fig = px.line(df, x='Quantile', y='Loss', markers=True, title='Modified Pinball Loss for Different Quantiles')
fig.update_layout(
    xaxis_title='Quantile',
    yaxis_title='Modified Pinball Loss',
    template='plotly_white'
)
fig.show()

In [40]:
import pickle
import plotly.express as px
import pandas as pd

# Pfad zu den Modellen
path = "D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/models/lgbr_model/models/i5_models/lgbr_q"
quantiles = [1, 2, 3, 4, 5, 6, 7, 8, 9]
predictions_dict = {}

# Berechnung der Vorhersagen für jedes Quantil
for i in quantiles:
    model_light = pickle.load(open(path + str(i) + ".pkl", 'rb'))
    predictions = model_light.predict(X_solar)
    predictions_dict[f'Quantile {i/10}'] = predictions

# Erstellen eines DataFrames für Plotly
df = pd.DataFrame(predictions_dict)
df['Actual'] = Y_solar.values

# Plotten der Vorhersagen und der tatsächlichen Werte mit Plotly
fig = px.line(df, title='Predictions and Actual Values for Different Quantiles')
for quantile in predictions_dict.keys():
    fig.add_scatter(x=df.index, y=df[quantile], mode='lines', name=quantile)
fig.add_scatter(x=df.index, y=df['Actual'], mode='lines', name='Actual', line=dict(color='black', width=2))
fig.update_layout(
    xaxis_title='Index',
    yaxis_title='Value',
    template='plotly_white'
)
fig.show()